In [ ]:
import os
import sys

INPUTFOLDER="../PAKs/cycle2/raw"
OUTPUTFOLDER="../PAKs/cycle2/extracted"

try:
    os.mkdir(OUTPUTFOLDER)
except FileExistsError:
    pass


menu=f"{INPUTFOLDER}/000001.nabu"
#list="../program-list.csv"

#offset = 0x97FA

with open(menu, "rb") as f:
    menu = bytearray(f.read())
    b_start = ''.join(f"{i:02X}" for i in menu[0x00:0x02])
    start = int.from_bytes(menu[0x00:0x02], byteorder='little')
    
    b_end = ''.join(f"{i:02X}" for i in menu[start:start+2])
    end = int.from_bytes(menu[start:start+2], byteorder='little')+start
    
    b_offset = ''.join(f"{i:02X}" for i in menu[start+10:start+12])
    offset = int.from_bytes(menu[start+10:start+12], byteorder='little')+start+0x19

    print(f"Boot: {b_start} ({start}) Start: {b_offset} ({offset}) End: {b_end} ({end})")

    list = []

    for x in range(offset, end, 0x1b):
        #print(f"{x}")
        item = menu[x:x+0x1b]    
        header = '|'.join(f"{i:02X}" for i in item[0:9])
        pakNumber = ''.join(f"{i:02X}" for i in item[6:9])
        name = item[9:0x1b].decode('ascii').strip()
        if (pakNumber == "000001"):
            continue
        list.append(f"{header} {pakNumber} {name}\n")
        
        filename = f"{INPUTFOLDER}/{pakNumber}.nabu"
        outFilename = f"{OUTPUTFOLDER}/{name}.nabu"
        rawFilename = f"{OUTPUTFOLDER}/{name}.pak"
        #print(F"{filename} -> {outFilename}")
        newProgram = bytearray(menu)
        writeStart = start+12
        readStart = 8
        
        try:
            with open(filename, "rb") as inFile:
                length = os.path.getsize(filename)-readStart
                programFile = inFile.read()

                with open(rawFilename, "wb") as rawFile:
                    rawFile.write(programFile)

                #print(f"{filename} {name}")    
                
                for i in range(0, length):
                    #print(f"Write: {writeStart+i} Read: {readStart+i}")
                    newProgram[writeStart+i] = programFile[readStart+i]
            with open(outFilename, "wb") as outFile:
                outFile.write(newProgram)
        except:
            pass

    listfile = f"{OUTPUTFOLDER}/program.list"
    with open(listfile, "w") as outFile:
        outFile.writelines(list)

    print("DONE")
    

    